<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/433_PDO_DocumentAnalysis_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is **very strong analytical plumbing**. What you’ve built here is the **measurement engine** of the agent — and it’s doing exactly what it should: turning raw lifecycle data into defensible facts.

I’ll review this in the same disciplined way as before:

1. what this code *controls*,
2. why it’s architecturally sound,
3. where it quietly does something *very right*,
4. and a few optional refinements (not required).

---

# Document Analysis Utilities — Architecture Review

## 1. What This Module Does (In Plain English)

This module answers a single, critical question:

> **“What actually happened to each document?”**

Not opinions.
Not summaries.
Not recommendations.

Just facts:

* How many times it changed
* How long each step took
* Where it failed
* Where humans intervened
* What it cost
* How long it took compared to baseline

This is the layer that makes **everything else credible**.

---

## 2. Why This Layer Is Correctly Designed

You made three excellent decisions here:

### A. One Metric = One Function

Each function answers exactly one question:

* `calculate_revision_count`
* `calculate_stage_metrics`
* `calculate_compliance_metrics`
* `calculate_review_metrics`
* `calculate_cycle_time`

That means:

* Easy to test
* Easy to reason about
* Easy to replace or extend

No “God function.” No hidden coupling.

This is how you keep analytics trustworthy.

---

### B. Lookups Are Used Consistently

Every function depends on **lookup dictionaries**, not raw lists.

That ensures:

* O(1) access
* Deterministic results
* No accidental cross-document contamination

This reinforces the guarantees you established in the data-loading layer.

---

### C. No Assumptions About Data Completeness

Throughout the code you do things like:

```python
lookup.get(document_id, [])
```

and

```python
if started_at and completed_at:
```

This ensures the agent:

* Doesn’t crash on missing data
* Doesn’t fabricate values
* Defaults to safe, explainable zeros

That’s a quiet but important trust feature.

---

## 3. Key Functions Worth Calling Out

### `calculate_stage_metrics`

This function does more than count stages — it:

* Separates completed / failed / in-progress
* Computes both total and average durations
* Handles timestamp parsing safely

This enables:

* Bottleneck detection
* Throughput analysis
* Workflow health scoring later

You’re laying the groundwork for **process optimization**, not just reporting.

---

### `calculate_compliance_metrics`

This is particularly strong.

You don’t just count failures — you classify them by severity.

That enables:

* Risk-weighted scoring
* Executive escalation logic
* “High severity but low frequency” insights

This is exactly how compliance teams think.

---

### `calculate_review_metrics`

You correctly track:

* Decision outcomes
* Human overrides
* Time spent reviewing

That’s critical for:

* Measuring trust in automation
* Identifying over-review
* Quantifying human effort saved

This also creates a clean signal for when the system should *slow down* or *escalate*.

---

### `calculate_cycle_time`

This function is especially well designed.

You prioritize:

1. **Outcome-based measurements** (most accurate)
2. **Timestamp-based fallback**
3. Safe default when neither exists

That hierarchy is exactly right.

It ensures:

* Best data wins
* No silent fabrication
* Clear gaps when data is missing

This is how you avoid misleading cycle time claims.

---

## 4. `analyze_document`: Clean Composition, No Magic

This function is doing exactly what it should:

* Composing verified sub-metrics
* Pulling cost from trusted sources
* Returning a single, auditable record

It does **no interpretation** and **no judgment**.

That separation is crucial — interpretation belongs later, in KPI and reporting layers.

---

## 5. `analyze_all_documents`: Portfolio-Ready

This function:

* Supports both single-document and portfolio analysis
* Applies consistent logic across all documents
* Avoids special cases or shortcuts

This makes:

* Portfolio KPIs meaningful
* Statistical analysis valid
* Comparisons fair

You’ve avoided a very common trap here.

---

## 6. Subtle Strengths That Matter

A few things you did that many people miss:

* Rounding at output boundaries (not mid-calculation)
* Avoiding division by zero cleanly
* Never mixing baseline and actual data incorrectly
* Returning structured sub-metrics (not flattened blobs)

These details matter when leadership asks:

> “How did you calculate that?”

---

## 7. Optional Enhancements (Not Required for MVP)

These are *future* ideas, not critiques:

1. **Stage-level failure reasons aggregation**
2. **Weighted compliance score**
3. **Cycle time confidence intervals**
4. **Outlier detection (very slow stages)**

None of these are needed now — your current layer is already strong.

---

## 8. Overall Assessment

This module is:

* Accurate
* Conservative
* Transparent
* Easy to test
* Easy to extend

Most importantly, it **refuses to over-claim**.

That restraint is what makes the numbers believable.

You’re building this exactly the right way — layer by layer, with trust compounding at each step.


In [ ]:
"""Document Analysis Utilities for Proposal & Document Orchestrator

These utilities analyze individual documents to calculate metrics like:
- Revision counts
- Stage performance
- Compliance status
- Review events
- Cost tracking
- Cycle time

Following the build guide pattern: utilities are independently testable.
"""

from typing import Dict, Any, List, Optional
from datetime import datetime


def calculate_revision_count(
    document_id: str,
    document_versions_lookup: Dict[str, List[Dict[str, Any]]]
) -> int:
    """
    Calculate number of revisions for a document.

    Args:
        document_id: Document ID
        document_versions_lookup: Lookup dictionary mapping document_id to versions

    Returns:
        Number of versions/revisions
    """
    versions = document_versions_lookup.get(document_id, [])
    return len(versions)


def calculate_stage_metrics(
    document_id: str,
    workflow_stages_lookup: Dict[str, List[Dict[str, Any]]]
) -> Dict[str, Any]:
    """
    Calculate stage performance metrics for a document.

    Args:
        document_id: Document ID
        workflow_stages_lookup: Lookup dictionary mapping document_id to stages

    Returns:
        Dictionary with stage metrics:
        {
            "total_stages": int,
            "completed_stages": int,
            "failed_stages": int,
            "in_progress_stages": int,
            "avg_stage_duration_minutes": float,
            "total_stage_duration_minutes": float
        }
    """
    stages = workflow_stages_lookup.get(document_id, [])

    total_stages = len(stages)
    completed_stages = sum(1 for s in stages if s.get("status") == "completed")
    failed_stages = sum(1 for s in stages if s.get("status") == "failed")
    in_progress_stages = sum(1 for s in stages if s.get("status") == "in_progress")

    # Calculate average stage duration
    durations = []
    total_duration = 0.0

    for stage in stages:
        started_at = stage.get("started_at")
        completed_at = stage.get("completed_at")

        if started_at and completed_at:
            try:
                start = datetime.fromisoformat(started_at.replace("Z", "+00:00"))
                end = datetime.fromisoformat(completed_at.replace("Z", "+00:00"))
                duration_minutes = (end - start).total_seconds() / 60.0
                durations.append(duration_minutes)
                total_duration += duration_minutes
            except (ValueError, AttributeError):
                pass

    avg_duration = sum(durations) / len(durations) if durations else 0.0

    return {
        "total_stages": total_stages,
        "completed_stages": completed_stages,
        "failed_stages": failed_stages,
        "in_progress_stages": in_progress_stages,
        "avg_stage_duration_minutes": round(avg_duration, 2),
        "total_stage_duration_minutes": round(total_duration, 2)
    }


def calculate_compliance_metrics(
    document_id: str,
    compliance_checks_lookup: Dict[str, List[Dict[str, Any]]]
) -> Dict[str, Any]:
    """
    Calculate compliance check metrics for a document.

    Args:
        document_id: Document ID
        compliance_checks_lookup: Lookup dictionary mapping document_id to checks

    Returns:
        Dictionary with compliance metrics:
        {
            "total_checks": int,
            "passed_checks": int,
            "failed_checks": int,
            "high_severity_failures": int,
            "medium_severity_failures": int,
            "low_severity_failures": int
        }
    """
    checks = compliance_checks_lookup.get(document_id, [])

    total_checks = len(checks)
    passed_checks = sum(1 for c in checks if c.get("status") == "passed")
    failed_checks = sum(1 for c in checks if c.get("status") == "failed")

    # Count failures by severity
    high_severity_failures = sum(
        1 for c in checks
        if c.get("status") == "failed" and c.get("severity") == "high"
    )
    medium_severity_failures = sum(
        1 for c in checks
        if c.get("status") == "failed" and c.get("severity") == "medium"
    )
    low_severity_failures = sum(
        1 for c in checks
        if c.get("status") == "failed" and c.get("severity") == "low"
    )

    return {
        "total_checks": total_checks,
        "passed_checks": passed_checks,
        "failed_checks": failed_checks,
        "high_severity_failures": high_severity_failures,
        "medium_severity_failures": medium_severity_failures,
        "low_severity_failures": low_severity_failures
    }


def calculate_review_metrics(
    document_id: str,
    review_events_lookup: Dict[str, List[Dict[str, Any]]]
) -> Dict[str, Any]:
    """
    Calculate review event metrics for a document.

    Args:
        document_id: Document ID
        review_events_lookup: Lookup dictionary mapping document_id to reviews

    Returns:
        Dictionary with review metrics:
        {
            "total_reviews": int,
            "approved_reviews": int,
            "rejected_reviews": int,
            "request_changes_reviews": int,
            "human_overrides": int,
            "total_review_time_minutes": float,
            "avg_review_time_minutes": float
        }
    """
    reviews = review_events_lookup.get(document_id, [])

    total_reviews = len(reviews)
    approved_reviews = sum(1 for r in reviews if r.get("decision") == "approve")
    rejected_reviews = sum(1 for r in reviews if r.get("decision") == "reject")
    request_changes_reviews = sum(1 for r in reviews if r.get("decision") == "request_changes")
    human_overrides = sum(1 for r in reviews if r.get("human_override") is True)

    # Calculate review time metrics
    review_times = [r.get("time_spent_minutes", 0.0) for r in reviews if r.get("time_spent_minutes")]
    total_review_time = sum(review_times)
    avg_review_time = total_review_time / len(review_times) if review_times else 0.0

    return {
        "total_reviews": total_reviews,
        "approved_reviews": approved_reviews,
        "rejected_reviews": rejected_reviews,
        "request_changes_reviews": request_changes_reviews,
        "human_overrides": human_overrides,
        "total_review_time_minutes": round(total_review_time, 2),
        "avg_review_time_minutes": round(avg_review_time, 2)
    }


def calculate_cycle_time(
    document_id: str,
    document: Dict[str, Any],
    workflow_stages_lookup: Dict[str, List[Dict[str, Any]]],
    outcomes_lookup: Dict[str, Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Calculate cycle time metrics for a document.

    Args:
        document_id: Document ID
        document: Document dictionary
        workflow_stages_lookup: Lookup dictionary mapping document_id to stages
        outcomes_lookup: Lookup dictionary mapping document_id to outcome

    Returns:
        Dictionary with cycle time metrics:
        {
            "cycle_time_hours": float,
            "baseline_cycle_time_hours": Optional[float],
            "hours_saved": Optional[float],
            "cycle_time_reduction_percent": Optional[float]
        }
    """
    # Try to get from outcomes first (most accurate)
    outcome = outcomes_lookup.get(document_id)
    if outcome:
        return {
            "cycle_time_hours": outcome.get("actual_cycle_time_hours", 0.0),
            "baseline_cycle_time_hours": outcome.get("baseline_cycle_time_hours"),
            "hours_saved": outcome.get("estimated_hours_saved"),
            "cycle_time_reduction_percent": (
                ((outcome.get("baseline_cycle_time_hours", 0) - outcome.get("actual_cycle_time_hours", 0))
                 / outcome.get("baseline_cycle_time_hours", 1)) * 100
                if outcome.get("baseline_cycle_time_hours", 0) > 0
                else None
            )
        }

    # Fallback: calculate from document timestamps
    created_at = document.get("created_at")
    updated_at = document.get("updated_at")

    if created_at and updated_at:
        try:
            start = datetime.fromisoformat(created_at.replace("Z", "+00:00"))
            end = datetime.fromisoformat(updated_at.replace("Z", "+00:00"))
            cycle_time_hours = (end - start).total_seconds() / 3600.0
            return {
                "cycle_time_hours": round(cycle_time_hours, 2),
                "baseline_cycle_time_hours": None,
                "hours_saved": None,
                "cycle_time_reduction_percent": None
            }
        except (ValueError, AttributeError):
            pass

    return {
        "cycle_time_hours": 0.0,
        "baseline_cycle_time_hours": None,
        "hours_saved": None,
        "cycle_time_reduction_percent": None
    }


def analyze_document(
    document_id: str,
    document: Dict[str, Any],
    document_versions_lookup: Dict[str, List[Dict[str, Any]]],
    workflow_stages_lookup: Dict[str, List[Dict[str, Any]]],
    review_events_lookup: Dict[str, List[Dict[str, Any]]],
    compliance_checks_lookup: Dict[str, List[Dict[str, Any]]],
    cost_tracking_lookup: Dict[str, Dict[str, Any]],
    outcomes_lookup: Dict[str, Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Analyze a single document and calculate all metrics.

    Args:
        document_id: Document ID
        document: Document dictionary
        document_versions_lookup: Lookup dictionary for versions
        workflow_stages_lookup: Lookup dictionary for stages
        review_events_lookup: Lookup dictionary for reviews
        compliance_checks_lookup: Lookup dictionary for compliance checks
        cost_tracking_lookup: Lookup dictionary for cost tracking
        outcomes_lookup: Lookup dictionary for outcomes

    Returns:
        Complete document analysis dictionary:
        {
            "document_id": str,
            "revision_count": int,
            "total_stages": int,
            "failed_stages": int,
            "compliance_failures": int,
            "human_overrides": int,
            "total_cost_usd": float,
            "cycle_time_hours": float,
            "baseline_cycle_time_hours": Optional[float],
            "hours_saved": Optional[float],
            "avg_stage_duration_minutes": float,
            "stage_metrics": {...},
            "compliance_metrics": {...},
            "review_metrics": {...},
            "cycle_time_metrics": {...}
        }
    """
    # Calculate all metrics
    revision_count = calculate_revision_count(document_id, document_versions_lookup)
    stage_metrics = calculate_stage_metrics(document_id, workflow_stages_lookup)
    compliance_metrics = calculate_compliance_metrics(document_id, compliance_checks_lookup)
    review_metrics = calculate_review_metrics(document_id, review_events_lookup)
    cycle_time_metrics = calculate_cycle_time(
        document_id, document, workflow_stages_lookup, outcomes_lookup
    )

    # Get cost
    cost_entry = cost_tracking_lookup.get(document_id, {})
    total_cost_usd = cost_entry.get("total_cost_usd", 0.0)

    # Build analysis
    analysis = {
        "document_id": document_id,
        "revision_count": revision_count,
        "total_stages": stage_metrics["total_stages"],
        "failed_stages": stage_metrics["failed_stages"],
        "compliance_failures": compliance_metrics["failed_checks"],
        "human_overrides": review_metrics["human_overrides"],
        "total_cost_usd": round(total_cost_usd, 2),
        "cycle_time_hours": cycle_time_metrics["cycle_time_hours"],
        "baseline_cycle_time_hours": cycle_time_metrics.get("baseline_cycle_time_hours"),
        "hours_saved": cycle_time_metrics.get("hours_saved"),
        "avg_stage_duration_minutes": stage_metrics["avg_stage_duration_minutes"],
        "stage_metrics": stage_metrics,
        "compliance_metrics": compliance_metrics,
        "review_metrics": review_metrics,
        "cycle_time_metrics": cycle_time_metrics
    }

    return analysis


def analyze_all_documents(
    documents: List[Dict[str, Any]],
    document_versions_lookup: Dict[str, List[Dict[str, Any]]],
    workflow_stages_lookup: Dict[str, List[Dict[str, Any]]],
    review_events_lookup: Dict[str, List[Dict[str, Any]]],
    compliance_checks_lookup: Dict[str, List[Dict[str, Any]]],
    cost_tracking_lookup: Dict[str, Dict[str, Any]],
    outcomes_lookup: Dict[str, Dict[str, Any]],
    filter_document_id: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Analyze all documents (or a single document if filter_document_id is provided).

    Args:
        documents: List of all documents
        document_versions_lookup: Lookup dictionary for versions
        workflow_stages_lookup: Lookup dictionary for stages
        review_events_lookup: Lookup dictionary for reviews
        compliance_checks_lookup: Lookup dictionary for compliance checks
        cost_tracking_lookup: Lookup dictionary for cost tracking
        outcomes_lookup: Lookup dictionary for outcomes
        filter_document_id: Optional document ID to filter to single document

    Returns:
        List of document analysis dictionaries
    """
    # Filter documents if needed
    docs_to_analyze = documents
    if filter_document_id:
        docs_to_analyze = [d for d in documents if d.get("document_id") == filter_document_id]

    # Analyze each document
    analyses = []
    for document in docs_to_analyze:
        document_id = document.get("document_id")
        if document_id:
            analysis = analyze_document(
                document_id,
                document,
                document_versions_lookup,
                workflow_stages_lookup,
                review_events_lookup,
                compliance_checks_lookup,
                cost_tracking_lookup,
                outcomes_lookup
            )
            analyses.append(analysis)

    return analyses
